In [1]:
from preprocessing import preprocessing_csr
import collections
import copy

In [2]:
class treeNode:
    """variables:
    name of the node, a count
    nodelink used to link similar items
    parent vaiable used to refer to the parent of the node in the tree
    node contains an empty dictionary for the children in the node"""
    
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = []

# get the child of this node with the name 'name'
    def get_child(self, name):
        for c in self.children:
            if c.name == name:
                return c
        
        return None
            
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, self.count)
        
        for child in self.children:
            child.disp(ind+1)

In [3]:
# testing the tree
rootNode = treeNode('pyramid',9,None)
rootNode.children.append(treeNode('eye',13,None))
rootNode.disp()

('  ', 'pyramid', 9)
('    ', 'eye', 13)


In [4]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    
    nodeToTest.nodeLink = targetNode

In [5]:
def updateTree(items, inTree, headerTable, count):
    """grow the Fp-tree with an itemset"""

    if inTree.get_child(items[0]) != None: #check if orderedItems[0] in retTree.children
#         inTree.children[items[0]].inc(count) #incrament count
        inTree.get_child(items[0]).inc(count)
    else:   #add items[0] to inTree.children
        inTree.children.append(treeNode(items[0], count, inTree))
        
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[len(inTree.children)-1] #W A R N I N G!
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[len(inTree.children)-1])
    
    if len(items) > 1: #call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.get_child(items[0]), headerTable, count)

In [65]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    """takes the dataset and the minimum support as arguments and builds the 
    FP-tree. This makes two passes through the dataset. The first pass goes 
    through everything in the dataset and counts the frequency of each term. 
    These are stored in the header table."""
    
    headerTable = {}
    
#     print 'dataset: ',len(dataSet)
    
    #go over dataSet twice
    for trans in dataSet:   #first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    
    print 'headerTable: ',headerTable

#     for k in list(headerTable):  #remove items not meeting minSup
#         if headerTable[k] < minSup: 
#             del(headerTable[k])

    freqItemSet = list(headerTable.keys())
#     print 'freqItemSet: ',freqItemSet

    if len(freqItemSet) == 0: 
        return None, None  #if no items meet min support -->get out
    
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    
#     print 'headerTable: ',headerTable

    retTree = treeNode('Null Set', 1, None) #create tree
    
    # dataSet.items() returns key-value tuple
#     print dataSet.items()
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = collections.OrderedDict()
        for item in tranSet:  #put transaction items in order
#             print item
#             if item in freqItemSet:
            localD[item] = headerTable[item][0]
#             t = item, headerTable[item][0]
#             localD.append(t)
#         print 'localD: ',localD
# SHOULD STUDY THIS PART!
        if len(localD) > 0:
#             orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
#             print orderedItems
            items = list(v[0] for v in localD.items())
#             print 'items: ', items
            updateTree(items, retTree, headerTable, count)   #populate tree with ordered freq itemset
            
    print 'headerTable: ',headerTable

    return retTree, headerTable #return tree and header table

In [7]:
def createInitSet(dataSet):
    """the createTree() function doesnt take the input data as lists. It expects
    a dictionary with the itemsets as the dictionary keys and the frequency as the
    value. A createInitSet() function does this conversion for you."""

    retDict = collections.OrderedDict()

    for trans in dataSet:
        if tuple(trans) in retDict:
            retDict[tuple(trans)] += 1
        else:
            retDict[tuple(trans)] = 1

    return retDict

In [12]:
def loadSimpDat():
    simpDat = preprocessing_csr.format()
#     simpDat = [['saan', 'pr', 'vb', 'abbreviation'], 
#                ['paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'], 
#                ['ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'],
#                ['ano', 'pr', 'description'], 
#                ['anong', 'nn', 'pr', 'description'], 
#                ['bakit', 'dt', 'nn', 'pr', 'description'],
#                ['sino', 'rb', 'pr', 'entity'], 
#                ['alin', 'cc', 'cd', 'dt', 'vb', 'pr', 'entity'], 
#                ['alin', 'rb', 'pr', 'dt', 'nn', 'pr', 'entity'],
#                ['alin', 'dt', 'vb', 'entity'], 
#                ['sino', 'dt', 'nn', 'cc', 'nn', 'pr', 'human'], 
#                ['sino', 'rb', 'pr', 'human'], 
#                ['alin', 'cc', 'nn', 'dt', 'vb', 'pr', 'human'],
#                ['saan', 'rb', 'pr', 'rb', 'location'], 
#                ['saan', 'pr', 'rb', 'location'], 
#                ['nasaan', 'pr', 'rb', 'rb', 'location'],
#                ['kailan', 'rb', 'pr', 'vb', 'numeric'], 
#                ['kailan', 'pr', 'nn', 'cc', 'vb', 'vb', 'nn', 'numeric'], 
#                ['kailan', 'pr', 'vb', 'vb', 'numeric'], 
#                ['alin', 'cc', 'pr', 'cc', 'vb', 'cc', 'nn', 'dt', 'nn', 'pr', 'numeric'], 
#                ['ano', 'dt', 'nn', 'pr', 'vb', 'numeric'], 
#                ['paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'], 
#                ['ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'],
#                ['ano', 'pr', 'description'], 
#                ['anong', 'nn', 'pr', 'description'], 
#                ['bakit', 'dt', 'nn', 'pr', 'description'],
#                ['sino', 'rb', 'pr', 'entity'], 
#                ['alin', 'cc', 'cd', 'dt', 'vb', 'pr', 'entity'], 
#                ['alin', 'rb', 'pr', 'dt', 'nn', 'pr', 'entity'],
#     ]
    return simpDat

In [66]:
#testing the sample data
simpDat = loadSimpDat()
# print 'sampleData: ',simpDat
# print '\n', len(simpDat)

In [67]:
initSet = createInitSet(simpDat)
# print initSet
# print '\n', len(initSet)

In [68]:
#The FP-tree
myFPtree, myHeaderTab = createTree(initSet, 1)
myFPtree.disp()

headerTable:  {'anong': 29, 'vb': 2415, 'kailang': 3, 'cc': 1408, 'ano': 405, 'paano': 443, 'numeric': 366, 'cd': 100, 'abbreviation': 16, 'paanong': 2, 'human': 494, 'pr': 2852, 'kailan': 440, 'nn': 2772, 'saan': 428, 'lm': 20, 'bakit': 446, 'location': 387, 'rb': 1656, 'saang': 6, 'alin-alin': 2, 'aling': 9, 'description': 1297, 'sino': 431, 'sinong': 7, 'jj': 553, 'nasaan': 8, 'dt': 2102, 'entity': 517, 'sinu-sino': 6, 'alin': 406, 'anu-ano': 5, 'inaano': 1}
headerTable:  {'anong': [29, <__main__.treeNode instance at 0x7f34a43d6440>], 'vb': [2415, <__main__.treeNode instance at 0x7f34a43d4290>], 'kailang': [3, <__main__.treeNode instance at 0x7f348e7be248>], 'cc': [1408, <__main__.treeNode instance at 0x7f34a43d0b90>], 'ano': [405, <__main__.treeNode instance at 0x7f34a449d290>], 'paano': [443, <__main__.treeNode instance at 0x7f34a440ebd8>], 'numeric': [366, <__main__.treeNode instance at 0x7f34a4451e60>], 'cd': [100, <__main__.treeNode instance at 0x7f34a40a9ab8>], 'abbreviation':

### Mine the fp-tree

In [40]:
def ascendTreeV2(leafNode, prefixPath): #ascends from leaf node to root
    """ which ascends the tree, collecting the names of items it encounters"""

    if leafNode.parent != None:
        prefixPath.append(leafNode)
        ascendTree(leafNode.parent, prefixPath)

In [57]:
def findPrefixPathV2(basePat, treeNode, minSup): #treeNode comes from header table
    """This function iterates through the linked list until it hits the end. 
    For each item it encounters, it calls ascendTree().This list is 
    returned and added to the conditional pattern base dictionary 
    called condPats."""

    condPats = {}
    
    while treeNode != None:
        prefixPath = []
        ascendTreeV2(treeNode, prefixPath)
        
        for node in prefixPath:
            if node.count < minSup:
                del prefixPath[:]
#                 print prefixPath
                break
        
        if len(prefixPath) > 1: 
            condPats[tuple(prefixPath[1:])] = treeNode.count
            
        treeNode = treeNode.nodeLink
        
    return condPats

In [55]:
def mineTreeV2(myFPtree, headerTable, minSup):
    suffixes = ['entity', 'abbreviation', 'description', 'human', 'location', 'numeric']
    
    
    for i in range(0, len(suffixes)):
        condPattBases = findPrefixPathV2(suffixes[i], headerTable[suffixes[i]][1], minSup)
        
        print 'Conditional FP-Tree for: ' ,suffixes[i], '\n', condPattBases
        

In [64]:
mineTreeV2(myFPtree, myHeaderTab, 2)

Conditional FP-Tree for:  entity 
{('nn', 'cc', 'jj', 'dt', 'alin'): 5, ('nn', 'pr', 'dt', 'cc', 'alin'): 2, ('pr', 'dt', 'rb', 'vb', 'ano'): 2, ('vb', 'cc', 'rb', 'anong'): 2, ('nn', 'dt', 'pr', 'alin'): 10, ('rb', 'dt', 'alin'): 7, ('vb', 'dt', 'pr', 'alin'): 22, ('vb', 'dt', 'ano'): 2, ('nn', 'pr', 'vb', 'dt', 'cc', 'alin'): 2, ('cc', 'pr', 'nn', 'dt', 'ano'): 3, ('vb', 'pr', 'dt', 'cc', 'alin'): 5, ('vb', 'jj', 'dt', 'pr', 'nn', 'cd', 'cc', 'rb', 'alin'): 2, ('pr', 'cc', 'dt', 'nn', 'anong'): 2, ('pr', 'sino'): 4, ('pr', 'vb', 'cd', 'cc', 'rb', 'alin'): 3, ('jj', 'dt', 'rb', 'alin'): 4, ('nn', 'cc', 'vb', 'dt', 'ano'): 3, ('vb', 'pr', 'nn', 'dt', 'cc', 'alin'): 6, ('pr', 'vb', 'dt', 'cd', 'cc', 'alin'): 2, ('nn', 'cc', 'jj', 'dt', 'ano'): 3, ('cc', 'nn', 'vb', 'dt', 'ano'): 4, ('nn', 'cc', 'jj', 'pr', 'vb', 'dt', 'ano'): 2, ('nn', 'pr', 'jj', 'anong'): 2, ('cc', 'nn', 'vb', 'dt', 'sino'): 2, ('jj', 'dt', 'alin'): 9, ('nn', 'vb', 'dt', 'ano'): 2, ('rb', 'pr', 'nn', 'dt', 'cc', 'alin